In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import shutil
import os

destination = "/content/food_project"

if os.path.exists(destination):
    shutil.rmtree(destination)

source = "/content/drive/MyDrive/Deep Learning/food_project"

shutil.copytree(source, destination)

print("Dataset copied successfully.")

Dataset copied successfully.


In [ ]:
import os
print(os.listdir("/content/food_project"))

['train', 'test', 'val']


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
base_path = "/content/food_project"

train_dataset = datasets.ImageFolder(base_path + "/train", transform=train_transforms)
val_dataset = datasets.ImageFolder(base_path + "/val", transform=val_test_transforms)
test_dataset = datasets.ImageFolder(base_path + "/test", transform=val_test_transforms)

print("Train size:", len(train_dataset))
print("Val size:", len(val_dataset))
print("Test size:", len(test_dataset))

print("Classes:", train_dataset.classes)

Train size: 3000
Val size: 3000
Test size: 3000
Classes: ['ice_cream', 'pizza', 'sushi']


In [ ]:
print("Train size:", len(train_dataset))
print("Val size:", len(val_dataset))
print("Test size:", len(test_dataset))
print("Classes:", train_dataset.classes)

Train size: 3000
Val size: 3000
Test size: 3000
Classes: ['ice_cream', 'pizza', 'sushi']


In [ ]:
import shutil
import os

base_path = "/content/food_project"

shutil.rmtree(base_path + "/val")
shutil.rmtree(base_path + "/test")

print("Val and Test deleted.")

Val and Test deleted.


In [ ]:
os.rename(base_path + "/train", base_path + "/raw")
print("Train renamed to raw.")

Train renamed to raw.


In [ ]:
print(os.listdir(base_path))

['raw']


In [ ]:
import os

base_path = "/content/food_project"
raw_path = base_path + "/raw"

classes = ["ice_cream", "pizza", "sushi"]

for split in ["train", "val", "test"]:
    for cls in classes:
        os.makedirs(os.path.join(base_path, split, cls), exist_ok=True)

print("Empty train/val/test folders created.")

Empty train/val/test folders created.


In [ ]:
import random
import shutil

for cls in classes:
    class_path = os.path.join(raw_path, cls)
    images = os.listdir(class_path)

    random.shuffle(images)

    total = len(images)
    train_end = int(0.7 * total)
    val_end = int(0.85 * total)

    train_imgs = images[:train_end]
    val_imgs = images[train_end:val_end]
    test_imgs = images[val_end:]

    for img in train_imgs:
        shutil.move(
            os.path.join(class_path, img),
            os.path.join(base_path, "train", cls, img)
        )

    for img in val_imgs:
        shutil.move(
            os.path.join(class_path, img),
            os.path.join(base_path, "val", cls, img)
        )

    for img in test_imgs:
        shutil.move(
            os.path.join(class_path, img),
            os.path.join(base_path, "test", cls, img)
        )

print("Dataset split completed.")

Dataset split completed.


In [ ]:
for split in ["train", "val", "test"]:
    for cls in classes:
        count = len(os.listdir(os.path.join(base_path, split, cls)))
        print(split, cls, count)

train ice_cream 700
train pizza 700
train sushi 700
val ice_cream 150
val pizza 150
val sushi 150
test ice_cream 150
test pizza 150
test sushi 150


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

base_path = "/content/food_project"

train_dataset = datasets.ImageFolder(base_path + "/train", transform=train_transforms)
val_dataset = datasets.ImageFolder(base_path + "/val", transform=val_test_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

print("Train:", len(train_dataset))
print("Val:", len(val_dataset))
print("Classes:", train_dataset.classes)

Train: 2100
Val: 450
Classes: ['ice_cream', 'pizza', 'sushi']


In [ ]:
model = models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 3)

model = model.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):

    model.train()
    train_loss = 0
    train_correct = 0
    train_total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_acc = train_correct / train_total

    model.eval()
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = val_correct / val_total

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Acc: {train_acc:.4f}")
    print(f"Val Acc: {val_acc:.4f}")
    print("-" * 30)

Epoch 1/5
Train Acc: 0.7367
Val Acc: 0.8800
------------------------------
Epoch 2/5
Train Acc: 0.8752
Val Acc: 0.8956
------------------------------
Epoch 3/5
Train Acc: 0.8995
Val Acc: 0.9111
------------------------------
Epoch 4/5
Train Acc: 0.9062
Val Acc: 0.9156
------------------------------
Epoch 5/5
Train Acc: 0.9181
Val Acc: 0.9044
------------------------------


In [ ]:
torch.save(model.state_dict(), "/content/food_model.pth")
print("Model saved successfully.")

Model saved successfully.


In [ ]:
test_dataset = datasets.ImageFolder(base_path + "/test", transform=val_test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

print("Test size:", len(test_dataset))

test_correct = 0
test_total = 0

model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_acc = test_correct / test_total
print("Test Accuracy:", test_acc)


Test size: 450
Test Accuracy: 0.8933333333333333


In [ ]:
test_dataset = datasets.ImageFolder(base_path + "/test", transform=val_test_transforms)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

print("Test size:", len(test_dataset))

Test size: 450


In [ ]:
test_correct = 0
test_total = 0

model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_acc = test_correct / test_total
print("Test Accuracy:", test_acc)

Test Accuracy: 0.8933333333333333


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving 5762.jpg to 5762.jpg


In [ ]:
class_names = train_dataset.classes

image_path = "5762.jpg"

prediction, confidence = predict_image(image_path, model, class_names)

print("Prediction:", prediction)
print("Confidence:", round(confidence * 100, 2), "%")

Prediction: ice_cream
Confidence: 91.38 %


In [ ]:
from PIL import Image
import torch.nn.functional as F

def predict_image(image_path, model, class_names):
    model.eval()

    image = Image.open(image_path).convert("RGB")
    image = val_test_transforms(image)
    image = image.unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        probabilities = F.softmax(outputs, dim=1)
        confidence, predicted = torch.max(probabilities, 1)

    predicted_class = class_names[predicted.item()]
    confidence = confidence.item()

    return predicted_class, confidence

In [ ]:
from google.colab import files
files.download("food_model.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>